# Quality control Groundwater

In [1]:
# necessary imports

import pandas as pd
import numpy as np

## Data exploration

In [2]:
df_groundwater_original = pd.read_csv('../../../../pygada/datasets/PFAS/gecombineerde_grondwater_dataset.csv', sep=';', decimal=',')
df_groundwater_original.head()

C:\Users\vandekgu\AppData\Local\Temp\ipykernel_13716\3495513427.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_groundwater_original = pd.read_csv('../../../../pygada/datasets/PFAS/gecombineerde_grondwater_dataset.csv', sep=';', decimal=',')


,Unnamed: 0,id,datum,x_m_L72,y_m_L72,top_m_mv,basis_m_mv,parameter,detectieconditie,meetwaarde,meeteenheid,bron
0,0,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFOSA,<,1.0,ng/l,VMM
1,1,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFOA,NaN,2.0,ng/l,VMM
2,2,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFDA,<,1.0,ng/l,VMM
3,3,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFOStotal,NaN,1.0,ng/l,VMM
4,4,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFBS,NaN,32.0,ng/l,VMM


In [3]:
# Translate the column names

df_groundwater_original = df_groundwater_original.rename(columns={'datum': 'date', 'detectieconditie': 'detection_condition', 'meetwaarde': 'value', 'meeteenheid': 'unit', 'bron': 'source'})
df_groundwater_original.head()

,Unnamed: 0,id,date,x_m_L72,y_m_L72,top_m_mv,basis_m_mv,parameter,detection_condition,value,unit,source
0,0,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFOSA,<,1.0,ng/l,VMM
1,1,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFOA,NaN,2.0,ng/l,VMM
2,2,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFDA,<,1.0,ng/l,VMM
3,3,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFOStotal,NaN,1.0,ng/l,VMM
4,4,861/61/2-F2/MPF2101,2021-07-27 00:00:00,143922.4688,214154.2188,4.5,5.0,PFBS,NaN,32.0,ng/l,VMM


In [4]:
# Get general information

df_groundwater_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160302 entries, 0 to 160301
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           160302 non-null  int64  
 1   id                   160302 non-null  object 
 2   date                 160302 non-null  object 
 3   x_m_L72              160302 non-null  float64
 4   y_m_L72              160302 non-null  float64
 5   top_m_mv             159394 non-null  float64
 6   basis_m_mv           159738 non-null  float64
 7   parameter            160302 non-null  object 
 8   detection_condition  158862 non-null  object 
 9   value                160302 non-null  float64
 10  unit                 160183 non-null  object 
 11  source               160302 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 14.7+ MB


Some attributes have missing data:

- top_m_mv
- basis_m_mv
- detection_condition
- unit

`detection_condition` and `unit` will be handled in the next steps.

When `top_m_mv` and/or `basis_m_mv` is empty these data records are dropped.


In [5]:
# Get the unique parameters

pd.set_option('display.max_rows', None)
df_groundwater_unique_parameters = df_groundwater_original.copy()
df_groundwater_unique_parameters_count = df_groundwater_unique_parameters.parameter.value_counts(dropna=False)
print(df_groundwater_unique_parameters_count)

PFOStotal                                   4482
PFOAtotal                                   4467
PFBS                                        4029
PFHxStotal                                  3802
PFUnDA                                      3776
PFHxA                                       3772
PFHpA                                       3771
PFDA                                        3767
PFDoDA                                      3764
PFBA                                        3763
PFNA                                        3760
PFTeDA                                      3725
PFDS                                        3722
PFTrDA                                      3722
PFHxDA                                      3721
PFODA                                       3707
PFOSAtotal                                  3653
6:2 FTS                                     3647
8:2 FTS                                     3646
4:2 FTS                                     3638
PFPeS               

The sum parameters are obtained based on the 'lower bound' principle. This implies that when a value is below the detection limit it is set as 0.
To not have this interpretation already made, these parameters are dropped from the dataset.

- EU DWRL-20
- EFSA-4
- PFAS Som kwantitatief
- PFAS Som indicatief
- PFAS totaal (Kwantitatief en indicatief)
- som PFAS kwantitatief
- som PFAS indicatief
- totaal PFAS

Some parameters are the same, but have different names.

- 'PFOAtotal' and 'som PFOA'
- 'PFOStotal' and 'som PFOS'
- 'PFOAbranched' and 'PFOA vertakt'
- 'PFOSbranched' and 'PFOS vertakt'
- 'PFOS lineair' and 'PFOS'
- 'PFOA lineair' and 'PFOA'
- 'DONA' and 'ADONA'

In [6]:
# Get the unique detection conditions

df_groundwater_unique_dc = df_groundwater_original.copy()
df_groundwater_unique_dc_count = df_groundwater_unique_dc.detection_condition.value_counts(dropna=False)
df_groundwater_unique_dc_count

<      118167
=       40684
NaN      1440
>          11
Name: detection_condition, dtype: int64

The NaN values corresponds to '='.

In [7]:
df_groundwater_unique_dc[df_groundwater_unique_dc['detection_condition']=='>']

,Unnamed: 0,id,date,x_m_L72,y_m_L72,top_m_mv,basis_m_mv,parameter,detection_condition,value,unit,source
22397,11580,31878970,2021-09-06,133724.50,206622.3,2.2,3.2,PFOStotal,>,11200.0,ng/l,OVAM
22401,11584,31878974,2021-09-06,133724.50,206622.3,2.2,3.2,PFOAtotal,>,803.0,ng/l,OVAM
22425,11608,31879034,2021-09-03,133812.80,206638.9,1.6,2.6,PFOAtotal,>,260.0,ng/l,OVAM
22469,11652,31879114,2021-09-03,133796.00,206638.7,3.0,4.0,PFOAtotal,>,398.0,ng/l,OVAM
22475,11658,31879120,2021-09-03,133796.00,206638.7,3.0,4.0,PFOStotal,>,153.0,ng/l,OVAM
22486,11669,31879131,2021-09-03,133844.23,206651.6,1.5,2.5,PFOAtotal,>,182.0,ng/l,OVAM
22505,11688,31879150,2021-07-16,133787.80,206664.0,3.0,4.0,PFOStotal,>,2020.0,ng/l,OVAM
22510,11693,31879155,2021-09-03,133787.80,206664.0,3.0,4.0,PFOStotal,>,2020.0,ng/l,OVAM
22514,11697,31879159,2021-09-03,133787.80,206664.0,3.0,4.0,PFOAtotal,>,273.0,ng/l,OVAM
22520,11703,31879165,2021-07-16,133787.80,206664.0,3.0,4.0,PFOAtotal,>,239.0,ng/l,OVAM


Drop these values for the moment.

In [8]:
# Get the unique units

df_groundwater_unique_unit = df_groundwater_original.copy()
df_groundwater_unique_unit_count = df_groundwater_unique_unit.unit.value_counts(dropna=False)
df_groundwater_unique_unit_count

ng/l         114849
µg/l          44915
onbenoemd       358
NaN             119
µg/kg ds         41
µg/kg            11
ng/kg ds          9
Name: unit, dtype: int64

Drop the unrealistic units for groundwater:

- onbenoemd
- NaN
- µg/kg ds
- µg/kg
- ng/kg ds

In [9]:
# Get the unique sources

df_groundwater_unique_source = df_groundwater_original.copy()
df_groundwater_unique_source_count = df_groundwater_unique_source.source.value_counts(dropna=False)
df_groundwater_unique_source_count

OVAM      143890
VMM        10817
Lantis      5595
Name: source, dtype: int64

In [10]:
# Get the unique top_m_mv

pd.set_option('display.max_rows', None)
df_groundwater_unique_top = df_groundwater_original.copy()
df_groundwater_unique_top_sorted = pd.DataFrame(df_groundwater_unique_top.top_m_mv.sort_values(ascending=False))
df_groundwater_unique_top_sorted = df_groundwater_unique_top_sorted.top_m_mv.value_counts(dropna=False)[df_groundwater_unique_top_sorted.top_m_mv.unique()]
print(df_groundwater_unique_top_sorted)

260.00       47
220.00       47
210.00       83
205.00       94
200.00       47
180.00       36
170.00       47
165.00       47
58.02        37
57.81        37
51.90        32
50.90       104
40.00       118
36.00       578
32.00        48
30.00       259
29.50        32
28.70        89
27.00        33
26.80        48
26.50        48
26.00        48
25.50        39
24.20        55
24.00       135
23.50       271
23.30        16
23.00       149
22.90        51
22.50       134
22.30        51
22.00       587
21.50        42
21.25        45
21.09       135
21.00       218
20.82        39
20.75        45
20.60        47
20.00        48
19.85        90
19.75        45
19.65        45
19.35        45
19.00        48
18.50        86
18.40        48
18.00       220
17.50        47
17.00       322
16.00       182
15.75        48
15.50       343
15.30        38
15.00       674
14.90       232
14.80       108
14.50        83
14.00      1197
13.50        48
13.30        36
13.20        48
13.10   

In [11]:
# Get the unique basis_m_mv

pd.set_option('display.max_rows', None)
df_groundwater_unique_basis = df_groundwater_original.copy()
df_groundwater_unique_basis_sorted = pd.DataFrame(df_groundwater_unique_basis.basis_m_mv.sort_values(ascending=False))
df_groundwater_unique_basis_sorted = df_groundwater_unique_basis_sorted.basis_m_mv.value_counts(dropna=False)[df_groundwater_unique_basis_sorted.basis_m_mv.unique()]
print(df_groundwater_unique_basis_sorted)

405.00       36
360.00       47
320.00       47
315.00       36
310.00       83
305.00       94
300.00       47
280.00       36
270.00       47
265.00       47
59.02        37
58.81        37
53.90       136
47.30        38
47.10        80
46.00       578
40.00       170
34.00        48
31.50        32
31.00        89
30.70        89
30.00        48
29.00        33
27.80        48
27.50        39
27.00        96
26.20        55
26.00        87
25.00       232
24.90        51
24.50       307
24.30        67
24.20        12
24.00       587
23.80        36
23.50       176
23.00       314
22.82        39
22.60        47
22.25        45
22.09       135
21.75        45
21.00        48
20.85        90
20.75        45
20.65        45
20.50        86
20.40        48
20.35        45
20.00       268
19.50        47
19.00        86
18.30        38
18.00       319
17.50        47
17.00       290
16.50       343
16.00       936
15.90       232
15.80       151
15.68        32
15.30        86
15.00   

## Data wrangling

In [12]:
df_groundwater_original_result = df_groundwater_original.copy()
df_groundwater_error = pd.DataFrame()
original_columns = df_groundwater_original.columns.values.tolist()
df_groundwater_error = pd.DataFrame(columns=original_columns)
df_error_count = []

# Transform date to datetime and remove unrealistic dates
# todo: adapt this snippet to no loop
for index, row in df_groundwater_original_result.iterrows():
    date=row['date']
    try:
        df_groundwater_original_result.at[index, 'date'] = pd.to_datetime(date, format='%Y-%m-%d %H:%M:%S').strftime("%Y/%m/%d")
    except ValueError:
        try:
             df_groundwater_original_result.at[index, 'date'] = pd.to_datetime(date, format='%Y/%m/%d').strftime("%Y/%m/%d")
        except ValueError:
            try:
                 df_groundwater_original_result.at[index, 'date'] = pd.to_datetime(date, format='%d/%m/%Y').strftime("%Y/%m/%d")
            except ValueError as e:
                row['error_date'] = 'Unrealistic date'
                df_row = row.to_frame(name='values').reset_index()
                df_row['original_index'] = index
                df_row = df_row.pivot(index='original_index', columns='index', values='values').reset_index()
                df_row = df_row.drop(columns=['original_index'])
                df_groundwater_error = df_groundwater_error.append(df_row)
                df_groundwater_original_result = df_groundwater_original_result.drop([index])
df_error_count.append(len(df_groundwater_error))

# Drop rows with sum parameters
df_groundwater_result_1 = df_groundwater_original_result[(~df_groundwater_original_result['parameter'].isin(['EU DWRL-20', 'EFSA-4', 'PFAS Som kwantitatief', 'PFAS Som indicatief', 'PFAS totaal (Kwantitatief en indicatief)', 'som PFAS kwantitatief', 'som PFAS indicatief', 'totaal PFAS']))]
df_groundwater_error_sum = df_groundwater_original_result[(df_groundwater_original_result['parameter'].isin(['EU DWRL-20', 'EFSA-4', 'PFAS Som kwantitatief', 'PFAS Som indicatief', 'PFAS totaal (Kwantitatief en indicatief)', 'som PFAS kwantitatief', 'som PFAS indicatief', 'totaal PFAS']))]
df_groundwater_error_sum['error_sum_parameter'] = 'Sum parameter'
df_groundwater_error = df_groundwater_error.merge(df_groundwater_error_sum, how='outer', on=original_columns)
df_error_count.append(len(df_groundwater_error_sum))

# Replace NaN detection_condition
df_groundwater_result_1['detection_condition'] = df_groundwater_result_1['detection_condition'].replace(np.nan, '=')

# Drop rows with higher than detection condition
df_groundwater_result_2 = df_groundwater_result_1[(~df_groundwater_result_1['detection_condition'].isin(['>']))]
df_groundwater_error_dc = df_groundwater_original_result[(df_groundwater_original_result['detection_condition'].isin(['>']))]
df_groundwater_error_dc['error_detection_condition'] = 'Higher than detection condition'
df_groundwater_error = df_groundwater_error.merge(df_groundwater_error_dc, how='outer', on=original_columns)
df_error_count.append(len(df_groundwater_error_dc))

# Drop rows with unrealistic units
df_groundwater_result_3 = df_groundwater_result_2[(~df_groundwater_result_2['unit'].isin(['onbenoemd', np.nan, 'µg/kg ds', 'µg/kg', 'ng/kg ds']))]
df_groundwater_error_unit = df_groundwater_original_result[(df_groundwater_original_result['unit'].isin(['onbenoemd', np.nan, 'µg/kg ds', 'µg/kg', 'ng/kg ds']))]
df_groundwater_error_unit['error_unrealistic_unit'] = 'Unrealistic unit'
df_groundwater_error = df_groundwater_error.merge(df_groundwater_error_unit, how='outer', on=original_columns)
df_error_count.append(len(df_groundwater_error_unit))

# Drop rows with empty `top_m_mv` and/or `basis_m_mv`
df_groundwater_result_4 = df_groundwater_result_3[(~df_groundwater_result_3['top_m_mv'].isna())]
df_groundwater_error_top = df_groundwater_original_result[(df_groundwater_original_result['top_m_mv'].isna())]
df_groundwater_error_top['error_top'] = 'No top m mv'
df_groundwater_error = df_groundwater_error.merge(df_groundwater_error_top, how='outer', on=original_columns)
df_error_count.append(len(df_groundwater_error_top))

df_groundwater_result_5 = df_groundwater_result_4[(~df_groundwater_result_4['basis_m_mv'].isna())]
df_groundwater_error_basis = df_groundwater_original_result[(df_groundwater_original_result['basis_m_mv'].isna())]
df_groundwater_error_basis['error_basis'] = 'No basis m mv'
df_groundwater_error = df_groundwater_error.merge(df_groundwater_error_basis, how='outer', on=original_columns)
df_error_count.append(len(df_groundwater_error_basis))

# Replace parameter values
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('som PFOA', 'PFOAtotal')
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('som PFOS', 'PFOStotal')
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('PFOA vertakt', 'PFOAbranched')
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('PFOS vertakt', 'PFOSbranched')
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('PFOS lineair', 'PFOS')
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('PFOA lineair', 'PFOA')
df_groundwater_result_5['parameter'] = df_groundwater_result_5['parameter'].replace('ADONA', 'DONA')

df_groundwater_result_5.to_csv('../../../../pygada/datasets/PFAS/combined_groundwater_dataset_cleaned.txt', encoding='utf-8-sig')
df_groundwater_error.to_csv('../../../../pygada/datasets/PFAS/combined_groundwater_dataset_error.txt', encoding='utf-8-sig')
df_error_count = pd.DataFrame({'Error':['Unrealistic date', 'sum parameter', 'detection condition', 'unrealistic unit', 'top', 'basis'], 'Count':df_error_count})

C:\Users\vandekgu\AppData\Local\Temp\ipykernel_13716\195033987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_groundwater_error = df_groundwater_error.append(df_row)
C:\Users\vandekgu\AppData\Local\Temp\ipykernel_13716\195033987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_groundwater_error = df_groundwater_error.append(df_row)
C:\Users\vandekgu\AppData\Local\Temp\ipykernel_13716\195033987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_groundwater_error = df_groundwater_error.append(df_row)
C:\Users\vandekgu\AppData\Local\Temp\ipykernel_13716\195033987.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_

In [13]:
# Clean dataset summary

pd.set_option('display.max_rows', None)
print(f'{df_groundwater_result_5.info()}\n\nClean dataset example\n{df_groundwater_result_5.head()}\n\nCount parameters\n{df_groundwater_result_5.parameter.value_counts(dropna=False)}\n\nCount detection conditions\n{df_groundwater_result_5.detection_condition.value_counts(dropna=False)}\n\nCount units\n{df_groundwater_result_5.unit.value_counts(dropna=False)}\n\nCount source\n{df_groundwater_result_5.source.value_counts(dropna=False)}')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151900 entries, 0 to 160301
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           151900 non-null  int64  
 1   id                   151900 non-null  object 
 2   date                 151900 non-null  object 
 3   x_m_L72              151900 non-null  float64
 4   y_m_L72              151900 non-null  float64
 5   top_m_mv             151900 non-null  float64
 6   basis_m_mv           151900 non-null  float64
 7   parameter            151900 non-null  object 
 8   detection_condition  151900 non-null  object 
 9   value                151900 non-null  float64
 10  unit                 151900 non-null  object 
 11  source               151900 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 15.1+ MB
None

Clean dataset example
   Unnamed: 0                   id        date      x_m_L72      y_m_L72  \
0       

In [14]:
# Error dataset summary

print(f'{df_groundwater_error.info()}\n\nError dataset example\n{df_groundwater_error.head()}\n\nCount source\n{df_groundwater_error.source.value_counts(dropna=False)}\n\nError count\n{df_error_count}')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8402 entries, 0 to 8401
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 8402 non-null   object
 1   id                         8402 non-null   object
 2   date                       8402 non-null   object
 3   x_m_L72                    8402 non-null   object
 4   y_m_L72                    8402 non-null   object
 5   top_m_mv                   7494 non-null   object
 6   basis_m_mv                 7838 non-null   object
 7   parameter                  8402 non-null   object
 8   detection_condition        8402 non-null   object
 9   value                      8402 non-null   object
 10  unit                       8283 non-null   object
 11  source                     8402 non-null   object
 12  error_date                 258 non-null    object
 13  error_sum_parameter        6764 non-null   object
 14  error_de